Read the file and split in X_train, X_test, y_train, y_test

In [1]:
# loading the Thyroid_Diff.csv dataset
import csv
import numpy as np

# to read the file into a python list
filename = "datasets/Thyroid_Diff.csv"
with open(filename, 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    data = [row for row in reader]

# this converts the list to an array of objects bc some columns are actually strings
data = np.array(data, dtype=object)  

# split features X from  labels y
X = data[:, :-1]  # all columns except last
y = data[:, -1]   # last column


In [2]:
X[0]

array(['27', 'F', 'No', 'No', 'No', 'Euthyroid',
       'Single nodular goiter-left', 'No', 'Micropapillary', 'Uni-Focal',
       'Low', 'T1a', 'N0', 'M0', 'I', 'Indeterminate'], dtype=object)

In [3]:
# converts string values into integers 
from sklearn.preprocessing import LabelEncoder

# encode X
for i in range(X.shape[1]):
    le = LabelEncoder()
    X[:, i] = le.fit_transform(X[:, i])

# convert to float to make sure can be used, 
# all values must be numbers before QSVC
X = X.astype(float)

# encode y
y_enc = LabelEncoder()
y = y_enc.fit_transform(y)


In [4]:
# basic printing and shape checking to confirm dataset size
print(X[0])
print("Shape X:",X.shape)
print("Shape y:", y.shape)

X_new = X[:][0:2]
print("Shape X:",X_new.shape)


[11.  0.  0.  0.  0.  2.  3.  3.  2.  1.  2.  0.  0.  0.  0.  2.]
Shape X: (383, 16)
Shape y: (383,)
Shape X: (2, 16)


In [5]:
# split into X_train, X_test, y_test, y_train
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Apply any of the tecniques from notebook 04_quantum_kernel in your dataset (QSVC, QClustering, QPCA)

In [6]:
# MODEL 1: QSVC + QuantumKernel (main baseline model)
from sklearn.utils import resample
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.kernels import FidelityStatevectorKernel
from qiskit_machine_learning.algorithms import QSVC

# reduce training set for speed
X_small, y_small = resample(X_train, y_train, n_samples=400, random_state=42)

# reduce test set
X_test_small = X_test[:100]
y_test_small = y_test[:100]

# use only first 6 features (6 qubits)
X_small = X_small[:, :11]
X_test_small = X_test_small[:, :11]

# simple feature map (fast)
feature_map = ZZFeatureMap(feature_dimension=X_small.shape[1], reps=1, entanglement="linear")

# build the kernel (no sampler argument in v0.8.4)
quantum_kernel = FidelityStatevectorKernel(feature_map=feature_map)

# train and evaluate QSVC
# this is the main quantum baseline model
# QSVC + optimal kernel

qsvc = QSVC(quantum_kernel=quantum_kernel)
qsvc.fit(X_small, y_small)

score = qsvc.score(X_test_small, y_test_small)
print(f"QSVC test accuracy: {score:.3f}")





QSVC test accuracy: 0.818


In [7]:
# this measures how long it takes to train the QSVC model
# DOES NOT AFFECT ACCURACY

import time
start = time.time()
qsvc.fit(X_small, y_small)
print("Training took", time.time() - start, "seconds")


Training took 2.5472989082336426 seconds


In [1]:
# MODEL 2: Classical SVC (RBF baseline)
# this is the normal, classical SVC
# this is a good baseline to compare how good the quantum methods are
from sklearn.svm import SVC

svc_rbf = SVC(kernel="rbf")
svc_rbf.fit(X_small, y_small)

score_rbf = svc_rbf.score(X_test_small, y_test_small)
print("Classical SVC (RBF) accuracy:", score_rbf)

NameError: name 'X_small' is not defined

In [10]:
# MODEL 3: Classical SVC + Quantum Kernel as a callable
# the classical SVC calls the quantum kernel during training

from sklearn.svm import SVC

svc_callable = SVC(kernel=quantum_kernel.evaluate)
svc_callable.fit(X_small, y_small)

score_callable = svc_callable.score(X_test_small, y_test_small)
print("Callable SVC + Quantum Kernel accuracy:", score_callable)


Callable SVC + Quantum Kernel accuracy: 0.8181818181818182


In [11]:
# MODEL 4: Classical SVC using precomputed Quantum Kernel
# SVC + QuantumTrainableKernel (Precomputed)
# the quantum kernel matrix is computed BEFORE training

# this is the quantum kernel matrix calculation
K_train = quantum_kernel.evaluate(X_small)
K_test  = quantum_kernel.evaluate(X_test_small, X_small)

svc_pre = SVC(kernel="precomputed")
svc_pre.fit(K_train, y_small)

score_pre = svc_pre.score(K_test, y_test_small)
print("Precomputed SVC + Quantum Kernel accuracy:", score_pre)


Precomputed SVC + Quantum Kernel accuracy: 0.8181818181818182


In [2]:
# MODEL 5: Trainable Quantum Kernel
# I think it is incompatible with the python version I have installed
# is not working, I believe it is incompatible 

from qiskit_machine_learning.kernels import TrainableFidelityStatevectorKernel
from qiskit_machine_learning.algorithms import QSVC
from sklearn.svm import SVC
from scipy.optimize import minimize
import numpy as np

# 1. Create trainable kernel
trainable_kernel = TrainableFidelityStatevectorKernel(feature_map=feature_map)

# 2. Initialize trainable parameters randomly
params = np.random.rand(len(trainable_kernel.get_trainable_parameters()))

# 3. Define a loss function for kernel training
def kernel_loss(params_vector):
    # Set kernel parameters
    trainable_kernel.set_parameters(params_vector)
    
    # Compute precomputed kernel matrix
    K_train = trainable_kernel.evaluate(X_small)
    
    # Fit SVC with precomputed kernel
    svc = SVC(kernel="precomputed")
    svc.fit(K_train, y_small)
    
    # Return 1 - accuracy as loss
    score = svc.score(K_train, y_small)
    return 1 - score

# 4. Optimize parameters using scipy's 'minimize'
res = minimize(kernel_loss, params, method='Powell', options={'maxiter': 20})

# 5. Set optimized parameters to the kernel
trainable_kernel.set_parameters(res.x)
optimal_kernel = trainable_kernel

# 6. Precompute kernel matrices
K_train_opt = optimal_kernel.evaluate(X_small)
K_test_opt  = optimal_kernel.evaluate(X_test_small, X_small)

# 7. Classical SVC with trained quantum kernel
svc_trainable = SVC(kernel="precomputed")
svc_trainable.fit(K_train_opt, y_small)
score_svc_trainable = svc_trainable.score(K_test_opt, y_test_small)
print("SVC + Trainable Quantum Kernel accuracy:", score_svc_trainable)

# 8. QSVC with trained quantum kernel
qsvc_trainable = QSVC(quantum_kernel=optimal_kernel)
qsvc_trainable.fit(X_small, y_small)
score_qsvc_trainable = qsvc_trainable.score(X_test_small, y_test_small)
print("QSVC + Trainable Quantum Kernel accuracy:", score_qsvc_trainable)


NameError: name 'feature_map' is not defined